# Rakamin Final Project - Kelompok 8

### Anggota Kelompok Infinity:
- Andre Adeputra S
- Gilang Muhammad Risky
- Jomas Sekar Pawestri
- M Nurkholis Fauzi
- Muhammad Syarif U
- Naomi Florenata Damanik
- Sakinah Nurul R
- Vanesa

## Import Libraries & Data Loading

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

pd.options.display.max_columns = None
pd.set_option('display.precision', 2)

In [3]:
df = pd.read_csv('data/Travel.csv')

## Stage 0: Pendahuluan

### Problem statement :
* Perusahaan Bhramanika Holidays di bidang Travel ingin membangun model bisnis yang layak untuk memperluas basis pelanggan. Salah satu cara yang akan dilakukan adalah dengan menawarkan jenis paket liburan yang baru. Namun, biaya pemasaran yang dikeluarkan cukup tinggi karena pelanggan dihubungi secara acak. Sedangkan berdasarkan data tahun lalu, conversion rate pelanggan yang membeli produk dari perusahaan hanya 18%. Oleh karena itu, perusahaan ini ingin menggunakan data yang tersedia untuk menyasar pelanggan yang potensial untuk menurunkan biaya pemasaran (marketing cost)

### Role :
* Kami dari Infinity Solutions merupakan agensi yang bergerak di bidang data untuk memberi solusi bisnis dari perusahaan klien. Peran kami di sini adalah untuk memberi rekomendasi kepada tim marketing dari klien melalui penggalian data-data marketing yang telah mereka lakukan sebelumnya.

### Goal : 
* Efisiensi target marketing

### Objective :
* Analisis data customer yang memungkinkan membeli paket perjalanan yang baru diperkenalkan 
* Menentukan fitur mana saja yang paling signifikan 
* Membuat machine learning untuk memprediksi pelanggan potensial
* Membuat rekomendasi target konsumen kepada tim marketing untuk diprioritaskan

### Business Metrics :
* Conversion Rate meningkat hingga 25% 


## Stage 1: EDA, Insight, dan Data Visualization